# Лабораторная работа 6. 
# Сетевые алгоритмы. Динамические алгоритмы поиска путей.


## Выполнил студент группы БВТ2004 Ким Игорь
***

### Задание

1.  Реализовать алгоритм поиска кратчайшего расстояния между двумя вершинами ориентированного взвешенного графа в соответствии с вариантом. 

2.  Предусмотреть задание графа в виде матрицы смежности/инцидентности, читаемой из файла, либо графически с помощью пользовательского интерфейса. 

3.  Разработать графический интерфейс пользователя с визуализацией графа и отображением кратчайшего расстояния между задаваемыми пользователем вершинами.

4. По результатам работы проанализировать временную сложность работы заданного алгоритма в зависимости от числа узлов и ребер графа.
Данные представить в виде таблицы.



### Алгоритмы:

Алгоритм Флойда-Уоршелла| Алгоритм Дейкстры | Алгоритм Беллмана-Форда | Алгоритм Джонсона| Алгоритм Левита | Алгоритм Йена



In [13]:
import ipycytoscape
import math
import copy
import time
from ipywidgets import Output
from IPython.display import display
from pprint import pformat
from heapq import *

In [25]:
def graph_from_str(str_graph: str):
    graph = [[weight for weight in row.split()] for row in str_graph.split('\n')] 
    
    for row in range(len(graph)):
        for col in range(len(graph)):
            if graph[row][col] == '-':
                graph[row][col] = math.inf
            else:
                graph[row][col] = int(graph[row][col])

    return graph

def read_test_graphs():
    file = open('exmpl_files/test_graphs.txt', 'r')
    file_data = file.read()
    file.close()
    
    str_graphs = file_data.split('\n////\n')
    graphs = []
    
    for str_graph in str_graphs:
        graphs.append(graph_from_str(str_graph))
        
    return graphs

def nodes_count(matrix: list):
    return len(matrix)

def edges_count(matrix: list):
    doubles = []
    cnt = 0
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if (matrix[i][j] != math.inf and i != j):
                doubles += [str(i+1) + str(j+1)]

    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if (matrix[i][j] != math.inf and i != j):
                if ((str(j+1) + str(i+1)) not in doubles):
                    cnt += 1
    return cnt
                    
def do_tests(graphs: list, fr: int, to: int, strategy: callable):
    results = []
    for gr in grphs:
        edges = edges_count(gr)
        nodes = nodes_count(gr)
        start_time = (time.perf_counter())
        strategy(gr, 1, 3)
        end_time = (time.perf_counter())
        results.append([edges, nodes, end_time - start_time])
    print_results(results)
        
def print_results(results: list):
    print ("{:<6} | {:<6} | {:<8}".format('edges', 'nodes', 'time(ms)'))
    for result in results:
        print ("{:<6} | {:<6} | {:<8}".format(result[0], result[1], result[2]*100))
        
def print_matrix(matrix: list[list[int]]):
    print('\n'.join([''.join(['{:8}'.format(str(item)) for item in row]) for row in matrix]))

In [7]:
class Heap(object):
    def __init__(self):
        self.index = 0
        self.__data = []

    def push(self, item, priority: int):
        heappush(self.__data, (priority, self.index, item))
        self.index += 1

    def pop(self):
        return heappop(self.__data)[2]

    def size(self) -> int:
        return len(self.__data)

    def __str__(self) -> str:
        return str(self.__data)

    def __repr__(self) -> str:
        return self.__str__()


In [10]:
class InteractiveGraph:
    def __init__(self, matrix = None, strat: callable = None):
        self.__cytoscapeobj = ipycytoscape.CytoscapeWidget()
        self.__out = Output()

        self.__selected_nodes = []
        self.__source = []
        self.__path = None
        
        if (matrix is not None):
            self.set_data(matrix)
        if (strat is not None):
            self.__strat = strat
            
        self.__init_styles()
        self.__init_events()
    
    def set_data(self, matrix):
        if (type(matrix) == list):
            matrix = self.__convert_data(matrix)
        self.__cytoscapeobj.graph.add_graph_from_json(matrix)
    
    def __convert_data(self, matrix: list):
        self.__source = matrix
        result = {
            'nodes' : [],
            'edges' : [],
        }
        nodes_count = len(matrix)

        for i in range(nodes_count):
            result['nodes'].append({'data': { 'id': i+1, 'name' : i+1}})
            
        doubles = []
        for i in range(nodes_count):
            for j in range(nodes_count):
                if (matrix[i][j] != math.inf and i != j):
                    doubles += [str(i+1) + str(j+1)]
                    
        for i in range(nodes_count):
            for j in range(nodes_count):
                if (matrix[i][j] != math.inf and i != j):
                    if ((str(j+1) + str(i+1)) not in doubles):
                        result['edges'].append({'data': { 'source': i+1, 'target': j+1, 'weight': matrix[i][j], 'arrow': 'bezier'}})
                    else:
                        result['edges'].append({'data': { 'source': i+1, 'target': j+1, 'weight': matrix[i][j]}})
        return result
    
    def set_strategy(self, strat: callable):
        self.__strat = strat
    
    def display(self):
        display(self.__cytoscapeobj)
        display(self.__out)
        
    def __init_events(self):
        def click_handler(node):
            if (node['data']['id'] not in self.__selected_nodes):
                if (len(self.__selected_nodes) == 2):
                    old = self.__selected_nodes.pop(0)
                    delt = self.__cytoscapeobj.graph.nodes[int(old) - 1]
                    delt.classes = ''
                    
                    self.__selected_nodes.append(node['data']['id'])
                    new = self.__cytoscapeobj.graph.nodes[int(node['data']['id']) - 1]
                    new.classes = 'highlighted'
                    
                    self.__reset_path()
                else:
                    self.__selected_nodes.append(node['data']['id'])
                    new = self.__cytoscapeobj.graph.nodes[int(node['data']['id']) - 1]
                    new.classes = 'highlighted'
            else:
                self.__selected_nodes.remove(node['data']['id'])
                delt = self.__cytoscapeobj.graph.nodes[int(node['data']['id']) - 1]
                delt.classes = ''
                self.__reset_path()
            if (len(self.__selected_nodes) == 2) :     
                    self.__calculate(int(self.__selected_nodes[0]), int(self.__selected_nodes[1]))
                    if (type(self.__path) == list):
                        self.__light_path()
                        
        self.__cytoscapeobj.on('node', 'click', click_handler)
    
    def __calculate(self, fr, to):
        self.__path = self.__strat(copy.deepcopy(self.__source), fr, to)
    
    def __light_path(self):
        for i in range(len(self.__path) - 1):
            src = self.__path[i]
            trg = self.__path[i + 1]
            for edge in self.__cytoscapeobj.graph.edges:
                if (edge.data['source'] == src and edge.data['target'] == trg or  edge.data['source'] == trg and edge.data['target'] == src):
                    edge.classes += 'highlighted'
    
    def __reset_path(self):
        for edge in self.__cytoscapeobj.graph.edges:
            edge.classes = ''
        self.__path = None
    
    def refresh(self):
        self.__init_styles()
        self.__init_events()
    
    def __init_styles(self):
        self.__cytoscapeobj.set_layout(name='cola')
        self.__cytoscapeobj.set_style(
                       [{
                          "selector":"node",
                          "style":{
                             'text-valign': 'center',
                             "height": 20,
                             "color" : "#ffffff",
                             "width": 20,
                             "background-color": "#11479e",
                             "content" : "data(name)"
                          }
                       },
                       {
                          "selector":"edge",
                          "style":{
                             "content" : 'data(weight)',
                             "width":5,
                             "opacity":0.5,
                             'line-color': '#9dbaea',
                             'target-arrow-shape': 'triangle',
                             'target-arrow-color': '#9dbaea',
                             'curve-style': 'data(arrow)'
                          }
                       },
                       {
                           "selector": "edge.highlighted",
                            "css":{
                                "line-color": "#008000",
                                'line-color': '#008000',
                                'target-arrow-shape': 'triangle',
                                'target-arrow-color': '#008000',
                                'curve-style': 'data(arrow)'
                            }
                       },
                       {
                           "selector": "node.highlighted",
                            "css": {
                                "background-color": "green",
                                'target-arrow-color': '#008000',
                            }
                       },
                       {
                           "selector": "edge.double",
                            "css": {
                                "line-color": "#008000",
                                'line-color': '#008000',
                            }
                       }
                    ])
grphs = read_test_graphs()        

### Выполнение:

In [37]:
# Алгоритм Флойда-Уоршелла
def floyd(graph: list[list[int]], start: int, end: int) -> list[int]:
    inf_graph = copy.deepcopy(graph)
    prev_graph = [[None for _ in range(len(graph))] for _ in range(len(graph))]
    start -= 1
    end -= 1
    
    for row in range(len(inf_graph)):
        for col in range(len(inf_graph)):
            if col == row:
                inf_graph[row][col] = 0

            if 0 < inf_graph[row][col] < math.inf:
                prev_graph[row][col] = row
                  
    for k in range(len(inf_graph)):
        for i in range(len(inf_graph)):
            for j in range(len(inf_graph)):
                if inf_graph[i][k] + inf_graph[k][j] < inf_graph[i][j]:    
                    inf_graph[i][j] = inf_graph[i][k] + inf_graph[k][j]
                    prev_graph[i][j] = prev_graph[k][j]
    
    if prev_graph[start][end] is None:
        return []
    
    path = []
    
    while end != None:
        path.append(end + 1)
        end = prev_graph[start][end]
        
    return list(reversed(path))

gr = InteractiveGraph()
gr.set_strategy(floyd)
gr.set_data(grphs[-1])
gr.display()

do_tests(grphs, 1, 3, floyd)

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'text-valig…

Output()

edges  | nodes  | time(ms)
6      | 5      | 0.012570001126732677
7      | 5      | 0.010479999764356762
8      | 5      | 0.01033000007737428
8      | 6      | 0.014950000331737101
9      | 6      | 0.01483000087318942
10     | 6      | 0.015290000010281801
11     | 8      | 0.02944999869214371
12     | 8      | 0.029919999360572547
13     | 8      | 0.03026999911526218


In [27]:
# Алгоритм Дейкстры
def deks(graph: list[list[int]], start: int, end: int) -> list[int]:
    start -= 1
    end -= 1
    prev_graph = [[None for _ in range(len(graph))] for _ in range(len(graph))]
    
    heap = Heap()
    heap.push(start, 0)
    came_from = {start: None}
    cost_so_far = {start: 0}
    
    while heap.size() > 0:
        current = heap.pop()
        
        if current == end:
            break
            
        neigbors = []
        
        for neighbor in range(len(graph[current])):
            if graph[current][neighbor] < math.inf:
                new_cost = cost_so_far[current] + graph[current][neighbor]
                if neighbor not in cost_so_far or new_cost < cost_so_far[neighbor]:
                    cost_so_far[neighbor] = new_cost
                    priority = new_cost
                    heap.push(neighbor, priority)
                    came_from[neighbor] = current
                    
    if current != end:
        return []
                
    path = []

    while current is not None:
        path.append(str(current + 1))
        current = came_from[current]
    path.reverse()
    path = [int(el) for el in path]
    return path
    
    
gr = InteractiveGraph()
gr.set_strategy(deks)
gr.set_data(grphs[-2])
gr.display()

do_tests(grphs, 1, 3, deks)

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'text-valig…

Output()

edges  | nodes  | time(ms)
6      | 5      | 0.0035999997635371983
7      | 5      | 0.0017400001524947584
8      | 5      | 0.001720000000204891
8      | 6      | 0.0016299993149004877
9      | 6      | 0.0015199999324977398
10     | 6      | 0.0015199999324977398
11     | 8      | 0.002340000355616212
12     | 8      | 0.002430001040920615
13     | 8      | 0.0023600005079060793


In [29]:
# Алгоритм Беллмана-Форда
def bellman_ford(graph: list[list[int]], start: int, end: int) -> list[int]:                
    start -= 1
    end -= 1
    dist = [math.inf] * len(graph)
    dist[start] = 0
    prevs = [None] * len(graph)
    
    for k in range(len(graph) - 1):
        for i in range(len(graph)):
            for j in range(len(graph)):
                if dist[i] < math.inf and dist[i] + graph[i][j] < dist[j]:
                    prevs[j] = i
                    dist[j] = dist[i] + graph[i][j]
                
    for i in range(len(graph)):
        for j in range(len(graph)):
            if dist[i] < math.inf and dist[i] + graph[i][j] < dist[j]:
                # Граф содержит цикл отрицательного веса
                return []
    
    if prevs[end] == None:
        return []
    
    path = []
            
    while end is not None:
        path.append(str(end + 1))
        end = prevs[end]
        
    path.reverse()
    path = [int(el) for el in path]
    
    return path

gr = InteractiveGraph()
gr.set_strategy(bellman_ford)
gr.set_data(grphs[-3])
gr.display()

do_tests(grphs, 1, 3, bellman_ford)

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'text-valig…

Output()

edges  | nodes  | time(ms)
6      | 5      | 0.006749999010935426
7      | 5      | 0.006059999577701092
8      | 5      | 0.006040000880602747
8      | 6      | 0.009709999721962959
9      | 6      | 0.01109000004362315
10     | 6      | 0.00957000011112541
11     | 8      | 0.020519999088719487
12     | 8      | 0.022119999630376697
13     | 8      | 0.03442000015638769


In [30]:
# Алгоритм Джонсона
def johns(graph: list[list[int]], start: int, end: int) -> list[int]:
    bf_graph = copy.deepcopy(graph)
    
    for row in range(len(bf_graph)):
        bf_graph[row].append(math.inf)
        
    bf_graph.append([0] * len(bf_graph))
    bf_graph[-1].append(math.inf)
    
    dist = [math.inf] * len(bf_graph)
    dist[-1] = 0
    
    for k in range(len(bf_graph) - 1):
        for i in range(len(bf_graph)):
            for j in range(len(bf_graph)):
                if dist[i] < math.inf and dist[i] + bf_graph[i][j] < dist[j]:
                    dist[j] = dist[i] + bf_graph[i][j]
                
    for i in range(len(bf_graph)):
        for j in range(len(bf_graph)):
            if dist[i] < math.inf and dist[i] + bf_graph[i][j] < dist[j]:
                # Граф содержит цикл отрицательного веса
                return []
            
    d_graph = copy.deepcopy(graph)
    
    for row in range(len(d_graph)):
        for col in range(len(d_graph)):
            d_graph[row][col] = d_graph[row][col] + dist[row] - dist[col]
          
    return deks(d_graph, start, end)

gr = InteractiveGraph()
gr.set_strategy(johns)
gr.set_data(grphs[-4])
gr.display()

do_tests(grphs, 1, 3, johns)

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'text-valig…

Output()

edges  | nodes  | time(ms)
6      | 5      | 0.03436999977566302
7      | 5      | 0.03040000010514632
8      | 5      | 0.030089999199844897
8      | 6      | 0.042019999818876386
9      | 6      | 0.056540001241955906
10     | 6      | 0.08202000026358292
11     | 8      | 0.07405999931506813
12     | 8      | 0.0701400000252761
13     | 8      | 0.11395999899832532


In [31]:
# Алгоритм Левита
def levit(graph: list[list[int]], start: int, end: int) -> list[int]:    
    start -= 1
    end -= 1
    
    dist = [math.inf] * len(graph)
    dist[start] = 0
    
    prevs = [None] * len(graph)
    
    ready = []
    processing = []
    waiting = []
    
    processing.append(start)
    
    for vertex in range(len(graph)):
        if vertex != start:
            waiting.append(vertex)
            
    while len(processing) > 0:
        vertex = processing.pop(0)
        ready.append(vertex)
        
        for neighbor in range(len(graph)):
            if graph[vertex][neighbor] < math.inf:
                if neighbor in waiting:
                    processing.append(neighbor)
                    waiting.remove(neighbor)
                    dist[neighbor] = dist[vertex] + graph[vertex][neighbor]
                    prevs[neighbor] = vertex
                    
                elif neighbor in processing:
                    if dist[vertex] + graph[vertex][neighbor] < dist[neighbor]:
                        dist[neighbor] = dist[vertex] + graph[vertex][neighbor]
                        prevs[neighbor] = vertex
                    
                elif neighbor in ready:
                    if dist[vertex] + graph[vertex][neighbor] < dist[neighbor]:
                        dist[neighbor] = dist[vertex] + graph[vertex][neighbor]
                        processing.insert(0, neighbor)
                        ready.remove(neighbor)
                        prevs[neighbor] = vertex
                                                
    path = []
            
    while end is not None:
        path.append(str(end + 1))
        end = prevs[end]
    
    path.reverse()
    path = [int(el) for el in path]
    
    return path

gr = InteractiveGraph()
gr.set_strategy(levit)
gr.set_data(grphs[-5])
gr.display()

do_tests(grphs, 1, 3, levit)

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'text-valig…

Output()

edges  | nodes  | time(ms)
6      | 5      | 0.002919998951256275
7      | 5      | 0.0020000006770715117
8      | 5      | 0.0030800001695752144
8      | 6      | 0.0021100000594742596
9      | 6      | 0.002069999754894525
10     | 6      | 0.008260000322479755
11     | 8      | 0.006269999721553177
12     | 8      | 0.004189999890513718
13     | 8      | 0.009539999882690609


In [36]:
# Алгоритм Йена
def path_length(path: list[int], graph: list[list[int]]):
    length = 0
    
    for i in range(1, len(path)):
        length += graph[int(path[i - 1]) - 1][int(path[i]) - 1]
    
    return length

def yen(graph: list[list[int]], start: int, end: int) -> list[int]:
    global k
    shortest = deks(graph, start, end)
    visited = [shortest]
    
    while k > 1:
        new_shortest = None
        new_shortest_length = math.inf
        
        for i in range(1, len(shortest)):
            new_graph = copy.deepcopy(graph)
            new_graph[int(shortest[i - 1]) - 1][int(shortest[i]) - 1] = math.inf
            
            path = deks(new_graph, start, end)
            path_len = path_length(path, new_graph)
            
            if len(path) > 1 and path_len < new_shortest_length and path not in visited:
                new_shortest_length = path_len
                new_shortest = path
                
        if new_shortest == None:
            return []
                
        shortest = new_shortest
        visited.append(shortest)
        k -= 1
    
    return shortest

k = int(input('Номер кратчайшего пути '))
gr = InteractiveGraph()
gr.set_strategy(yen)
gr.set_data(grphs[-5])
gr.display()

do_tests(grphs, 1, 3, yen)

Номер кратчайшего пути 1


CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'text-valig…

Output()

edges  | nodes  | time(ms)
6      | 5      | 0.0036299999919719994
7      | 5      | 0.0016899997717700899
8      | 5      | 0.0017099999240599573
8      | 6      | 0.0015899990103207529
9      | 6      | 0.0015700003132224083
10     | 6      | 0.0015300000086426735
11     | 8      | 0.0024199995095841587
12     | 8      | 0.0025000001187436283
13     | 8      | 0.0023999993572942913
